In [ ]:
import sys
sys.path.insert(0,'/home/pi/zumi/lib')
import Speaker as speaker
import Engine as engine
from IPython import display 
import numpy as np
import PIL.Image 
from TawnCam import PiCamera
import time
import cv2

import TouristDemoHelper as tourist_demo

model = tourist_demo.load_model("landmark")
# speaker.play()

def take_a_bunch_of_pictures(robot, camera, label):
    cnt = 50
    while cnt >= 0:
        print(cnt)
        cnt = cnt - 1
        image = camera.run()
        image = cv2.flip(image, -1)

        if label in ['up', 'left', 'right', 'eiffel', 'start', 'chicago', 'nyc', 'china', 'bigben', 'khalifa', 'seattle', 'intersection']:
            file_name = "/home/pi/zumi/sample/deep-learning-demos/tourist/images/" + robot + "." + str(time.time()) + "." + label + ".jpg"
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            cv2.imwrite(file_name, image)


In [ ]:
which_robot = "jumi"

camera = PiCamera(image_w=64, image_h=64, image_d=3, framerate=10)

try:
    while True:

        frame = camera.run()
        frame = cv2.flip(frame, -1)
        
        pred = model.predict(frame[None, :, :, :])
        iArrowDir = np.argmax(pred[0])

        if iArrowDir == 0:
            predicted_label = 'up' 
        elif iArrowDir == 1:
            predicted_label = 'left'
        elif iArrowDir == 2:
            predicted_label = 'right' 
        elif iArrowDir == 3:
            predicted_label = "nyc"
        elif iArrowDir == 4:
            predicted_label = "seattle"
        elif iArrowDir == 5:
            predicted_label = "eiffel"
        elif iArrowDir == 6:
            predicted_label = "bigben"
        elif iArrowDir == 7:
            predicted_label = "china"
        elif iArrowDir == 8:
            predicted_label = "khalifa"
        elif iArrowDir == 9:
            predicted_label = "chicago"
        elif iArrowDir == 10:
            predicted_label = "intersection"
        elif iArrowDir == 11:
            predicted_label = "start"
                    
        display.clear_output(wait=True)
        display.display(PIL.Image.fromarray(frame))
        display.display(predicted_label)
        display.display(iArrowDir)
        
        display.display(pred)
        
        isCorrect = input("Input y if correct: ") 
        if isCorrect == 'y':
            correct_label = predicted_label
        else:
            correct_label = input("what is the correct label? ")
            take_a_bunch_of_pictures(which_robot, camera, correct_label)
                                       
        if correct_label == "up":
            engine.forward_a_bit()
        elif correct_label == 'left':
            engine.left_a_bit()
        elif correct_label == "right":
            engine.right_a_bit()
        else:
            engine.forward_a_bit()
        
        engine.stop()
    
finally:
    engine.stop()
    camera.shutdown()